In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

INFILE = '../data/comparison.csv'

In [ ]:
df = pd.read_csv(INFILE)
df.head()

In [ ]:
# select participants who passed comprehension checks
df = df[(df.ParticipantID!='test')]
print('N total participants', len(pd.unique(df.ParticipantID)))
df = df[(df.FcastComprehension==1) & (df.BonusComprehension==1)]
print('N passed comprehension check', len(pd.unique(df.ParticipantID)))
# select 'actual' forecasts (as opposed to practice forecasts)
df = df[df.Practice==0]
# scale forecasts to be between 0 and 1 (as opposed to 0 and 100)
df['Fcast'] = df['Fcast'] / 100.
# compute DVs
df['Score'] = (df.y - df.Fcast)**2 - (df.y - df.output)**2
df['dFPP'] = (df.Fcast - df.output).loc[(df.y==0)]
df.values.shape

In [ ]:
# not pre-registered cell: added to remove empty values from Score column
df = df.dropna(subset=['Score'])
df.values.shape

In [ ]:
# prediction performance hypothesis
# positive coef on constant regressor indicates model outperformed human predictions
reg = sm.OLS(df.Score, np.array([1]*len(df)))
res = reg.fit().get_robustcov_results(cov_type='cluster', groups=df.ParticipantID)
res.summary()

In [ ]:
# fairness hypothesis
# positive coef on black indicates that human predictions are less fair than model predictions
no_recid_df = df[df.y == 0]
reg = sm.OLS(no_recid_df.Score, no_recid_df.black)
res = reg.fit().get_robustcov_results(cov_type='cluster', groups=no_recid_df.ParticipantID)
res.summary()

In [ ]:
# compute bonuses
completed_df = df[df.Status == 'Completed']
bonus_df = (1-completed_df.groupby(by='ParticipantID').Score.mean())
bonus_df.to_csv('../data/comparison_bonus_1.csv')